# 7. LangSmith を使った RAG アプリケーションの評価


In [18]:
import os
# from google.colab import userdata
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 7.4. Ragas による合成テストデータの生成


### パッケージのインストール


In [1]:
!pip install langchain-core==0.2.30 langchain-openai==0.1.21 \
    langchain-community==0.2.12 GitPython==3.1.43 \
    langchain-chroma==0.1.2 chromadb==0.5.3 \
    ragas==0.1.14 nest-asyncio==1.6.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 4.7 MB/s eta 0:00:00
  Using cached langchain_community-0.2.12-py3-none-any.whl (2.3 MB)
  Using cached langchain_chroma-0.1.2-py3-none-any.whl (9.3 kB)
  Using cached chromadb-0.5.3-py3-none-any.whl (559 kB)
  Using cached ragas-0.1.14-py3-none-any.whl (163 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00:00:0100:01
  Using cached chroma_hnswlib-0.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached pysbd-0.3.4-py3-none-any.whl (71 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.9 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━

### 検索対象のドキュメントのロード


In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

384


### Ragas による合成テストデータ生成の実装


In [4]:
for document in documents:
    document.metadata["filename"] = document.metadata["source"]

#### 【注意】既知のエラーについて

以下のコードで gpt-4o を使用すると OpenAI API の Usage tier 次第で RateLimitError が発生することが報告されています。

OpenAI API の Usage tier については公式ドキュメントの以下のページを参照してください。

https://platform.openai.com/docs/guides/rate-limits/usage-tiers

このエラーが発生した場合は、以下のどちらかの対応を実施してください。

1. 同じ Tier でも gpt-4o よりレートリミットの高い gpt-4o-mini を使用する
   - この場合、生成される合成テストデータの品質は低くなることが想定されます
2. 課金などにより Tier を上げる
   - Tier 2 で RateLimitError が発生しないことを確認済みです (2024 年 10 月 31 日時点)


In [ ]:
# testset.csvがある場合はコメントアウト（料金結構かかるから）

import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    # generator_llm=ChatOpenAI(model="gpt-4o"),
    generator_llm=ChatOpenAI(model="gpt-4o-mini"),
    # critic_llm=ChatOpenAI(model="gpt-4o"),
    critic_llm=ChatOpenAI(model="gpt-4o-mini"),
    embeddings=OpenAIEmbeddings(),
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25},
)

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating: 100%|██████████| 4/4 [00:12<00:00,  3.08s/it]           


In [ ]:
display(testset.to_pandas())

# 一回保存
testset.to_pandas().to_csv('testset.cdv')

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What models does GigaChat LLM provide?,[# Salute Devices\n\nSalute Devices provides G...,GigaChat LLM provides models for chat and embe...,simple,[{'source': 'docs/docs/integrations/providers/...,True
1,What is the purpose of the chat model in the C...,[# Cohere\n\n>[Cohere](https://cohere.ai/about...,The purpose of the chat model in the Cohere in...,simple,[{'source': 'docs/docs/integrations/providers/...,True
2,What services does Naver offer with AI and cloud?,[# NAVER\n\nAll functionality related to `Nave...,Naver offers a comprehensive suite of cloud se...,reasoning,[{'source': 'docs/docs/integrations/providers/...,True
3,What's a Python snippet for using HyperCLOVA X...,[# NAVER\n\nAll functionality related to `Nave...,A Python snippet for using HyperCLOVA X in Nav...,multi_context,[{'source': 'docs/docs/integrations/providers/...,True


### LangSmith の Dataset の作成


In [9]:
from langsmith import Client

dataset_name = "agent-book"

client = Client()

if client.has_dataset(dataset_name=dataset_name):
    client.delete_dataset(dataset_name=dataset_name)

dataset = client.create_dataset(dataset_name=dataset_name)

### 合成テストデータの保存


In [ ]:
inputs = []
outputs = []
metadatas = []

for testset_record in testset.test_data:
    inputs.append(
        {
            "question": testset_record.question,
        }
    )
    outputs.append(
        {
            "contexts": testset_record.contexts,
            "ground_truth": testset_record.ground_truth,
        }
    )
    metadatas.append(
        {
            "source": testset_record.metadata[0]["source"],
            "evolution_type": testset_record.evolution_type,
        }
    )

In [11]:
client.create_examples(
    inputs=inputs,
    outputs=outputs,
    metadata=metadatas,
    dataset_id=dataset.id,
)

## 7.5. LangSmith と Ragas を使ったオフライン評価の実装


### カスタム Evaluator の実装


In [12]:
from typing import Any

from langchain_core.embeddings import Embeddings
from langchain_core.language_models import BaseChatModel
from langsmith.schemas import Example, Run
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from ragas.metrics.base import Metric, MetricWithEmbeddings, MetricWithLLM


class RagasMetricEvaluator:
    def __init__(self, metric: Metric, llm: BaseChatModel, embeddings: Embeddings):
        self.metric = metric

        # LLMとEmbeddingsをMetricに設定
        if isinstance(self.metric, MetricWithLLM):
            self.metric.llm = LangchainLLMWrapper(llm)
        if isinstance(self.metric, MetricWithEmbeddings):
            self.metric.embeddings = LangchainEmbeddingsWrapper(embeddings)

    def evaluate(self, run: Run, example: Example) -> dict[str, Any]:
        context_strs = [doc.page_content for doc in run.outputs["contexts"]]

        # Ragasの評価メトリクスのscoreメソッドでスコアを算出
        score = self.metric.score(
            {
                "question": example.inputs["question"],
                "answer": run.outputs["answer"],
                "contexts": context_strs,
                "ground_truth": example.outputs["ground_truth"],
            },
        )
        return {"key": self.metric.name, "score": score}

In [13]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from ragas.metrics import answer_relevancy, context_precision

metrics = [context_precision, answer_relevancy]

llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

evaluators = [
    RagasMetricEvaluator(metric, llm, embeddings).evaluate
    for metric in metrics
]

### 推論の関数の実装


In [14]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [15]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [16]:
def predict(inputs: dict[str, Any]) -> dict[str, Any]:
    question = inputs["question"]
    output = chain.invoke(question)
    return {
        "contexts": output["context"],
        "answer": output["answer"],
    }

### オフライン評価の実装・実行


In [20]:


import nest_asyncio
nest_asyncio.apply()

from langsmith.evaluation import evaluate

evaluate(
    predict,
    data="agent-book",
    evaluators=evaluators,
)

View the evaluation results for experiment: 'elderly-cheese-14' at:
https://smith.langchain.com/o/64d2ddf6-fc4f-4ab5-98d6-5a5e505bb610/datasets/715f3375-7106-446b-a306-fb26447008a7/compare?selectedSessions=0d75c567-015c-4551-a477-900dbea78189




0it [00:00, ?it/s]Error running evaluator <DynamicRunEvaluator evaluate> on run 53af9879-9752-49f5-8f8d-603398401aaa: APIConnectionError('Connection error.')
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.10/site-packages/openai/_base_client.py", line 1576, in _request
    response = await self._client.send(
  File "/home/user/.local/lib/python3.10/site-packages/httpx/_client.py", line 1629, in send
    response = await self._send_handling_auth(
  File "/home/user/.local/lib/python3.10/site-packages/httpx/_client.py", line 1657, in _send_handling_auth
    response = await self._send_handling_redirects(
  File "/home/user/.local/lib/python3.10/site-packages/httpx/_client.py", line 1694, in _send_handling_redirects
    response = await self._send_single_request(request)
  File "/home/user/.local/lib/python3.10/site-packages/httpx/_client.py", line 1730, in _send_single_request
    response = await transport.handle_async_request(request)
  File "/home/user/.local

,inputs.question,outputs.contexts,outputs.answer,error,reference.contexts,reference.ground_truth,feedback.context_precision,feedback.evaluate,execution_time,example_id,id,feedback.answer_relevancy
0,What services does Naver offer with AI and cloud?,[page_content='# NAVER\n\nAll functionality re...,Naver offers a range of services related to AI...,None,[# NAVER\n\nAll functionality related to `Nave...,Naver offers a comprehensive suite of cloud se...,1.0,NaN,3.099704,ed4f4d55-f07e-43b3-b0c9-b7b1726c161f,53af9879-9752-49f5-8f8d-603398401aaa,NaN
1,What's a Python snippet for using HyperCLOVA X...,[page_content='# NAVER\n\nAll functionality re...,"To use HyperCLOVA X in Naver Cloud, you can us...",None,[# NAVER\n\nAll functionality related to `Nave...,A Python snippet for using HyperCLOVA X in Nav...,1.0,NaN,2.125752,b5132b1c-cf7a-455f-8a83-c2f81a042cc5,14ebc50d-a7ef-460f-b36c-aec168e4a33d,NaN
2,What models does GigaChat LLM provide?,[page_content='# Salute Devices\n\nSalute Devi...,"GigaChat LLM provides chat models, LLMs (Large...",None,[# Salute Devices\n\nSalute Devices provides G...,GigaChat LLM provides models for chat and embe...,1.0,NaN,1.159430,e2f4a563-bd5d-42ab-854f-eac8cb0a780a,0ffc6d99-2687-4b6d-90f9-22840325f9da,0.899094
3,What is the purpose of the chat model in the C...,[page_content='# Cohere\n\n>[Cohere](https://c...,The purpose of the chat model in the Cohere in...,None,[# Cohere\n\n>[Cohere](https://cohere.ai/about...,The purpose of the chat model in the Cohere in...,1.0,NaN,1.923903,57def430-7fb8-4f82-a50c-9bffb329a78e,05a765e7-b880-4b9e-ae43-312cec5369ca,1.000000


## LangSmith を使ったオンライン評価の実装


### フィードバックボタンを表示する関数の実装


In [22]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # GoodボタンとBadボタンを準備
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # クリックされた際に実行される関数を定義
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("フィードバックを送信しました")

    # ボタンがクリックされた際にon_button_clicked関数を実行
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # ボタンを表示
    display(good_button, bad_button)

### フィードバックボタンを表示


In [23]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("LangChainの概要を教えて")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。

1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。

2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。

LangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装しており、数百のプロバイダーと統合されています。また、複数のオープンソースライブラリで構成されており、ユーザーは自分のニーズに応じてさまざまなコンポーネントを選択して使用できます。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())